In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [24]:
df=pd.read_csv("data\\stud.csv")

In [25]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [26]:
X=df.drop('math_score',axis=1)

In [28]:
X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [29]:
y=df['math_score']

In [30]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [35]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns

In [37]:
print("Numerical features:",num_features)
print("Categorical Features:",cat_features)

Numerical features: Index(['reading_score', 'writing_score'], dtype='object')
Categorical Features: Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')


In [45]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

In [49]:
categorical_transformer=OneHotEncoder()
numerical_transformer=StandardScaler()
preprocessor=ColumnTransformer([
('onehotencoder',categorical_transformer,cat_features),
('numericaltransformer',numerical_transformer,num_features)
])

In [50]:
X=preprocessor.fit_transform(X)

In [51]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [52]:
X.shape

(1000, 19)

In [53]:
from sklearn.model_selection import train_test_split

In [55]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42)

In [58]:
print(X_train.shape)
print(X_test.shape)
print("-------------------")
print(y_train.shape)
print(y_test.shape)

(800, 19)
(200, 19)
-------------------
(800,)
(200,)


In [59]:
def Evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [84]:
models={
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbour-Regressor":KNeighborsRegressor(),
    "DecissonTree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "CatBoosting Regressor":CatBoostRegressor(verbose=False),
    "Adaboost Regressor":AdaBoostRegressor()
    }
model_list=[]
r2_list=[]


In [85]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    train_model_mae,train_model_rmse,train_model_r2=Evaluate_model(y_train,y_train_pred)

    test_model_mae,test_model_rmse,test_model_r2=Evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model performance for Training Dataset:")
    print("Root Mean Squared Error :{:.4f}".format(train_model_rmse))
    print("Mean Absolute Error :{:.4f}".format(train_model_mae))
    print("R2 Score :{:.4f}".format(train_model_r2))


    print("Model performance for Test Dataset:")
    print("Root Mean Squared Error :{:.4f}".format(test_model_rmse))
    print("Mean Absolute Error :{:.4f}".format(test_model_mae))
    print("R2 Score :{:.4f}".format(test_model_r2))
    r2_list.append(test_model_r2)

Linear Regression
Model performance for Training Dataset:
Root Mean Squared Error :5.3231
Mean Absolute Error :4.2667
R2 Score :0.8743
Model performance for Test Dataset:
Root Mean Squared Error :5.3940
Mean Absolute Error :4.2148
R2 Score :0.8804
Lasso
Model performance for Training Dataset:
Root Mean Squared Error :6.5938
Mean Absolute Error :5.2063
R2 Score :0.8071
Model performance for Test Dataset:
Root Mean Squared Error :6.5197
Mean Absolute Error :5.1579
R2 Score :0.8253
Ridge
Model performance for Training Dataset:
Root Mean Squared Error :5.3233
Mean Absolute Error :4.2650
R2 Score :0.8743
Model performance for Test Dataset:
Root Mean Squared Error :5.3904
Mean Absolute Error :4.2111
R2 Score :0.8806
K-Neighbour-Regressor
Model performance for Training Dataset:
Root Mean Squared Error :5.7091
Mean Absolute Error :4.5175
R2 Score :0.8554
Model performance for Test Dataset:
Root Mean Squared Error :7.2583
Mean Absolute Error :5.6370
R2 Score :0.7835
DecissonTree
Model performan

In [86]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880433
7,CatBoosting Regressor,0.851632
5,Random Forest Regressor,0.850544
8,Adaboost Regressor,0.849596
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbour-Regressor,0.783497
4,DecissonTree,0.730581
